In [1]:
import os
import sys
import warnings

warnings.filterwarnings('ignore')

project_root_path_list = os.getcwd().split(os.sep)
project_root_path = os.sep.join(
    project_root_path_list[0:project_root_path_list.index("Tallary")+1])
sys.path.append(project_root_path)

# User handler

In [ ]:
import json
from pydantic import BaseModel, Field
from pydantic.config import ConfigDict

class BaseTools(BaseModel):
    model_config = ConfigDict(populate_by_name=True)

    def to_dict(self):
        return json.loads(self.model_dump_json(by_alias=True, exclude_unset=True))

class UpdateUser(BaseTools,BaseModel):
    userName:str = Field(None)
    password:str = Field(None)



In [ ]:
from pydantic import BaseModel
from abc import ABC, abstractmethod
from typing import Any, Dict, Mapping, Tuple, List
from sqlalchemy.exc import IntegrityError, SQLAlchemyError

from api_backend.handlers.logers.loger_handlers import LogerHandler
from api_backend.handlers.db.db_handlers import AbstractDataBaseHandler
from api_backend.handlers.db.orm_models.abstract_models import AbstractUsers


# from logers.loger_handlers import LogerHandler
# from db.db_handlers import AbstractDataBaseHandler
# from db.orm_models.abstract_models import AbstractUsers


class AbstractUserHandler(ABC):
    @abstractmethod
    def __init__(self, logerHandler, dbHandler, dbt):
        super().__init__()
        self.logerHandler:LogerHandler = logerHandler
        self.dbHandler:AbstractDataBaseHandler = dbHandler
        self.dbt:AbstractUsers = dbt 

    @abstractmethod
    def get_data(self):
        pass
    
    @abstractmethod
    def insert_data(self):
        pass
    
    @abstractmethod
    def delete_data(self):
        pass
    
    @abstractmethod
    def update_data(self):
        pass

class UserHandler(AbstractUserHandler):
    def __init__(self, logerHandler, dbHandler, dbt):
        super().__init__(logerHandler, dbHandler, dbt)

    async def get_data(self, columnFilters:List):
        return await self.dbHandler.get_table_data([self.dbt], columnFilters)

    async def insert_data(self, userName:str, password:str):
        createUserResponse = await self.dbHandler.insert_data(
            data=(self.dbt(
                userName=userName,
                password=password
                ),
            )
        )
        return createUserResponse
    
    async def delete_data(self, userID:int):
        return await self.dbHandler.delete_data(self.dbt, (self.dbt.id == userID,))

    @staticmethod
    def _to_updates_dict(upd: Any) -> Dict[str, Any]:
        if isinstance(upd, Mapping):
            return dict(upd)
        if isinstance(upd, BaseModel):
            return upd.model_dump(exclude_unset=True, exclude_none=True)
        return dict(upd)

    def __normalize_updates_for_model(self, updatesData: Dict[str, Any]) -> Tuple[Dict[str, Any], Dict[str, str]]:
        updatesData: Dict[str, Any] = self._to_updates_dict(updatesData)

        cols = {c.name: c for c in self.dbt.__table__.columns}
        valid: Dict[str, Any] = {}
        skipped: Dict[str, str] = {}

        for key, raw_val in updatesData.items():
            col = cols.get(key)
            if col is None:
                skipped[key] = "unknown column"
                continue
            if col.primary_key:
                skipped[key] = "primary key is not updatable"
                continue

            try:
                valid[key] = self.__coerce(
                    raw_val, col.type, nullable=col.nullable)
            except Exception as e:
                skipped[key] = f"conversion error: {e}"

        return valid, skipped

    async def update_data(self, userID:int, updatesData: Dict[str, Any]):
        async with self.dbHandler.create_session()() as sess:
            try:
                obj = await sess.get(self.dbt, userID)  # AsyncSession.get
                if obj is None:
                    return None
                valid, _ = self.__normalize_updates_for_model(updatesData)
                for k, v in valid.items():
                    setattr(obj, k, v)
                await sess.commit()
                await sess.refresh(obj)
                return obj
            except (IntegrityError, SQLAlchemyError):
                await sess.rollback()
                raise




In [ ]:
from api_backend.handlers.users.user import UserHandler
from api_backend.handlers.users.schema import UpdateUser
from api_backend.handlers.db.orm_models.sqlite_models import Users
from api_backend.handlers.db.db_handlers import SqliteHandlerAsync
from api_backend.handlers.logers.loger_handlers import LogerHandler


# logerHandler = LogerHandler(logerName="")
logerHandler = None
dbHandler = SqliteHandlerAsync(url="sqlite+aiosqlite:///database/database_draft.db")

userHandler = UserHandler(dbHandler=dbHandler, dbt=Users, logerHandler=logerHandler)

In [ ]:
insertData = await userHandler.insert_data(userName="user 1",password="admin")

In [ ]:
data = await userHandler.get_data(columnFilters=[])
for item in data:
    print(item.to_dict())

In [ ]:
dataDelete = await userHandler.delete_data(userID=1)
dataDelete

In [ ]:
updateUserData = UpdateUser(password="qwerty1")
dataUpdate = await userHandler.update_data(userID=4, updatesData = updateUserData)
dataUpdate.to_dict()

# User Service

In [ ]:
from fastapi import Depends
from abc import ABC, abstractmethod
from api_backend.handlers.users.schema import UpdateUser
from api_backend.handlers.users.user import UserHandler
from pydantic import BaseModel


class AuthUser(BaseModel):
    userName:str = Field()
    password:str = Field() 


class AbstractUserService(ABC):
    
    @abstractmethod
    def __init__(self, logerHandler, userHandler):
        super().__init__()
        self.logerHandler:LogerHandler = logerHandler
        self.userHandler: UserHandler = userHandler

    @abstractmethod
    def auth_user(self):
        pass
        
    @abstractmethod
    def get_users(self):
        pass
    
    @abstractmethod
    def create_user(self):
        pass

    @abstractmethod
    def update_user(self):
        pass

    @abstractmethod
    def delete_user(self):
        pass

class UserService(AbstractUserService):
    
    def __init__(self, logerHandler, userHandler):
        super().__init__(logerHandler, userHandler)

    async def auth_user(self, auth:AuthUser = Depends()):
        filter_ = (
            self.userHandler.dbt.userName == auth.userName,
            self.userHandler.dbt.password == auth.password,
        )
        data = await self.userHandler.get_data(columnFilters = filter_)
        dataLenth = data.__len__()
        
        if dataLenth == 1:
            return auth
        elif dataLenth > 1:
            raise
        elif dataLenth == 0:
            raise
        else:
            raise
    
    async def get_users(self,userID:int):
        return {"status":201}
        
    async def create_user(self):
        return {"status":201}

    async def update_user(self):
        return {"status":201}

    async def delete_user(self):
        return {"status":201}




# auth

In [ ]:
class BaseServiceAuthorizationHandler(ABC):
    @abstractmethod
    def __init__(self,auth):
        self.auth: BaseServerAuthorizationHandler = auth

    @abstractmethod
    def check_permissions_get(self):
        pass

    @abstractmethod
    def check_permissions_post(self):
        pass

    @abstractmethod
    def check_permissions_patch(self):
        pass

    @abstractmethod
    def check_permissions_delete(self):
        pass

# Alfa bank handler

In [ ]:
from api_backend.handlers.bank_files.bank_load_handlers import AlfaBankHandler
from api_backend.handlers.bank_files.schema import AlfaHandlerUpdateData
from api_backend.handlers.db.orm_models.sqlite_models import AlfaFinancialTransactions
from api_backend.handlers.db.db_handlers import SqliteHandlerAsync
from api_backend.handlers.logers.loger_handlers import LogerHandler

from api_backend.handlers.bank_files.bank_file_preprocessing import AlfaPreprocessingDataFileHandler 

# logerHandler = LogerHandler(logerName="")
logerHandler = None
logerHandler2 = None
dbHandler = SqliteHandlerAsync(url="sqlite+aiosqlite:///database/database_draft.db")

alfaPreprocessingDataFileHandler = AlfaPreprocessingDataFileHandler(
    logerHandler=logerHandler2,
)

alfaBankHandler = AlfaBankHandler(dbHandler=dbHandler, 
                              dbt=AlfaFinancialTransactions, 
                              logerHandler=logerHandler,
                              preprocessingHandler=alfaPreprocessingDataFileHandler)

alfaBankHandler

In [ ]:
from datetime import date

In [ ]:
insertedData = await alfaBankHandler.insert_data(
    userID=2, 
    fileName="abstract_test_file.csv",
    operationDate = date(2025,1,1),
    postingDate = date(2025,1,1),
    code = "transaction_test_code",
    category="Трусы",
    description="Трусы от десантницы маруси",
    currencyAmount=-148.8,
    status=None
)

insertedData

In [ ]:
insertedFile = await alfaBankHandler.insert_file(
    userID=1, 
    filePath= "handlers\\bank_files\\report_file_catalog\\alfa\\01 октября 2025 - 31 октября 2025.xlsx"
)

insertedFile

In [ ]:
deleteFilter = (alfaBankHandler.dbt.id == 148,) 
deletedData = await alfaBankHandler.delete_data(deleteFilter)
deletedData

In [ ]:
from api_backend.handlers.bank_files.schema import AlfaHandlerUpdateData

updateData = AlfaHandlerUpdateData(
    category="Одежда и обувь"
)
updatedData = await alfaBankHandler.update_data(148,updateData)
updatedData

In [ ]:
updatedData.to_dict()

# Tinkoff bank handler

In [ ]:
from api_backend.handlers.bank_files.bank_load_handlers import TinkoffBankHandler
from api_backend.handlers.bank_files.schema import AlfaHandlerUpdateData
from api_backend.handlers.db.orm_models.sqlite_models import TinkoffFinancialTransactions
from api_backend.handlers.db.db_handlers import SqliteHandlerAsync
from api_backend.handlers.logers.loger_handlers import LogerHandler

from api_backend.handlers.bank_files.bank_file_preprocessing import TinkoffPreprocessingDataFileHandler

# logerHandler = LogerHandler(logerName="")
logerHandler = None
logerHandler2 = None
dbHandler = SqliteHandlerAsync(url="sqlite+aiosqlite:///database/database_draft.db")

tinkoffPreprocessingDataFileHandler = TinkoffPreprocessingDataFileHandler(
    logerHandler=logerHandler2,
)

tinkoffBankHandler = TinkoffBankHandler(dbHandler=dbHandler, 
                              dbt=TinkoffFinancialTransactions, 
                              logerHandler=logerHandler,
                              preprocessingHandler=tinkoffPreprocessingDataFileHandler)

tinkoffBankHandler

In [ ]:
from datetime import date

In [ ]:
insertingData = await tinkoffBankHandler.insert_data(
    userID=2,
    fileName="abstract_test_file.csv",
    operationDate = date(2025,1,1),
    postingDate = date(2025,1,1),
    description2 = "Трусы",
    amount=-148.8,
    description="Трусы от десантницы маруси",
    currencyAmount=-148.8,
)

insertingData

In [ ]:
insertingFile = await tinkoffBankHandler.insert_file(
    userID=2,
    filePath='handlers\\bank_files\\report_file_catalog\\tinkoff\\pdf\\Выписка (1).pdf'
)

insertingFile

In [ ]:
from api_backend.handlers.bank_files.schema import TinkoffHandlerUpdateData

TinkoffHandlerUpdateData = TinkoffHandlerUpdateData(
    currencyAmount = 148.8,
    amount = 148.8
)

updatedData = await tinkoffBankHandler.update_data(transactionID=1,
                                                     updatesData=TinkoffHandlerUpdateData)

updatedData.to_dict()

In [ ]:
deleteFilter = (tinkoffBankHandler.dbt.amount < 1000,)
deletedData = await tinkoffBankHandler.delete_data(deleteFilter)
deletedData

# Bank Registry

In [ ]:
from api_backend.handlers.bank_files.bank_load_handlers import AlfaBankHandler,TinkoffBankHandler
from api_backend.handlers.bank_files.schema import AlfaHandlerUpdateData
from api_backend.handlers.db.orm_models.sqlite_models import AlfaFinancialTransactions, TinkoffFinancialTransactions
from api_backend.handlers.db.db_handlers import SqliteHandlerAsync
from api_backend.handlers.logers.loger_handlers import LogerHandler

from api_backend.handlers.bank_files.bank_file_preprocessing import AlfaPreprocessingDataFileHandler, TinkoffPreprocessingDataFileHandler

# logerHandler = LogerHandler(logerName="")
logerHandler = None
logerHandler2 = None
logerHandler3 = None
logerHandler4 = None


dbHandler = SqliteHandlerAsync(url="sqlite+aiosqlite:///database/database_draft.db")

alfaPreprocessingDataFileHandler = AlfaPreprocessingDataFileHandler(logerHandler=logerHandler2,)
tinkoffPreprocessingDataFileHandler = TinkoffPreprocessingDataFileHandler(logerHandler=logerHandler3,)

alfaBankHandler = AlfaBankHandler(dbHandler=dbHandler, 
                              dbt=AlfaFinancialTransactions, 
                              logerHandler=logerHandler,
                              preprocessingHandler=alfaPreprocessingDataFileHandler)


tinkoffBankHandler = TinkoffBankHandler(dbHandler=dbHandler, 
                              dbt=TinkoffFinancialTransactions, 
                              logerHandler=logerHandler4,
                              preprocessingHandler=tinkoffPreprocessingDataFileHandler)


In [8]:
from api_backend.handlers.bank_files.bank_registry import BankHandlerRegistry


registry = BankHandlerRegistry()
registry.register("alfa", alfaBankHandler)
registry.register("tinkoff", tinkoffBankHandler)


In [12]:
slug = 'tinkoff'
getDataFilter = (registry.get_handler(slug).dbt.userID == 2,)
bankData = await registry.get_handler(slug).get_data(getDataFilter)
bankData

# categorys

In [4]:
from handlers.db.db_handlers import SqliteHandlerAsync
from services.category.category import СategoryService
from handlers.db.orm_models.sqlite_models import CastomCategorysCatalog,CastomCategorysConditions,TinkoffFinancialTransactions,AlfaFinancialTransactions
from handlers.castom_category.category_catalog_handler import TransactionCategoryCatalogHandler
from handlers.castom_category.category_conditions_handler import TransactionCategoryConditionsHandler

from handlers.bank_files.bank_load_handlers import AlfaBankHandler, TinkoffBankHandler
from handlers.bank_files.bank_registry import RegistryConstSchema

alfaBankHandler = AlfaBankHandler(dbHandler=dbHandler,
                                  dbt=AlfaFinancialTransactions,
                                  logerHandler=logerHandler,
                                  preprocessingHandler=alfaPreprocessingDataFileHandler)

tinkoffBankHandler = TinkoffBankHandler(dbHandler=dbHandler,
                                        dbt=TinkoffFinancialTransactions,
                                        logerHandler=logerHandler,
                                        preprocessingHandler=tinkoffPreprocessingDataFileHandler)

bankRegistry = BankHandlerRegistry()

alfaHandlerConfig = RegistryConstSchema(fileStorageDir=os.sep.join(["handlers","bank_files","report_file_catalog","alfa"]))
bankRegistry.register("alfa", alfaBankHandler, alfaHandlerConfig)
tinkoffHandlerConfig = RegistryConstSchema(fileStorageDir=os.sep.join(["handlers","bank_files","report_file_catalog","tinkoff","pdf"]))
bankRegistry.register("tinkoff", tinkoffBankHandler, tinkoffHandlerConfig)


logerHandler = None
dbHandler = SqliteHandlerAsync(url="sqlite+aiosqlite:///database/database_draft.db")


transactionCategoryHandler = TransactionCategoryCatalogHandler(dbHandler=dbHandler, 
                                                               dbt=CastomCategorysCatalog, 
                                                               logerHandler=logerHandler)
transactionCategoryConditionsHandler = TransactionCategoryConditionsHandler(dbHandler=dbHandler, 
                                                                            dbt=CastomCategorysConditions, 
                                                                            logerHandler=logerHandler)


categoryService = СategoryService(
        categoryCatalogHandler=transactionCategoryHandler,
        categoryConditionsHandler=transactionCategoryConditionsHandler,
        bankRgistry=bankRegistry,
        logerHandler=logerHandler)

ImportError: attempted relative import beyond top-level package

In [2]:
from initialization import categoryService

ImportError: attempted relative import with no known parent package

In [1]:
data = await categoryService.get_transactions(['alfa'])

NameError: name 'categoryService' is not defined